In [20]:
import os
import glob
import math
import numpy as np
import pandas as pd

In [9]:
matrix = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/HeLa_cell/combined_results/results/MatrixEQTL_result/hg19_SNP.csv"
df_m = pd.read_csv(matrix)
df_m = df_m.set_index(["id"])
df_m.head()
selected_list, index_list = [], []
for index, values in df_m.iterrows():
    values = values.tolist()
    if (values.count(0) > 1) and (values.count(1) > 1) and (values.count(2) > 1):
        selected_list.append(values)
        index_list.append(index)
df_geno = pd.DataFrame(selected_list, columns=df_m.columns, index=index_list)
print(df_geno.head())

             SRR5925271-  SRR5925272-  SRR5925287-  SRR5925288-  SRR5925299-  \
chr1:91515             0            1            0            0            0   
chr1:91536             0            1            0            0            0   
chr1:91581             0            1            0            0            0   
chr1:122872            0            0            2            1            2   
chr1:158006            0            0            1            0            2   

             SRR5925300-  SRR6714636-  SRR6714637-  SRR6714640-  SRR6714641-  \
chr1:91515             0            0            0            2            0   
chr1:91536             0            0            0            2            0   
chr1:91581             0            0            0            2            0   
chr1:122872            1            0            0            0            0   
chr1:158006            1            0            0            2            0   

             SRR6714644-  SRR6714645- 

In [13]:
mapping = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/HeLa_cell/combined_results/results/validation_result/m6AVar/m6AQTL_cis_linear_filter.txt"
df_map = pd.read_table(mapping, sep="\t", header=None)
df_map.head()
####
index_list = index_list # SNP
methy_list = [] # m6A
map_dict = {}
####
for index, values in df_map.iterrows():
    if values[0] in index_list:
        methy_list.append(values[1])
        map_dict[values[0]] = [values[1], values[2]]
methy_list = list(set(methy_list))

In [14]:
methy = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/HeLa_cell/combined_results/results/MatrixEQTL_result/hg19_quantification.matrix"
df_methy = pd.read_table(methy, sep="\t", index_col=0)
# print(df_methy.head())
df_pheno = df_methy[df_methy.index.isin(methy_list)]
print(df_pheno.head())

                           SRR5925271-  SRR5925272-  SRR5925287-  SRR5925288-  \
id                                                                              
chr14:100757429-100759901     2.973713     4.523478     4.772248     3.806934   
chr2:85062924-85065374        1.262625     2.101578     1.155902     1.155795   
chr21:46353613-46354147       1.645968     2.220213     3.364202     2.682206   
chr7:1607061-1607583          1.491703     2.429535     4.004795     3.298926   
chr1:227152636-227153237      1.239343     1.769380     1.956890     1.744201   

                           SRR5925299-  SRR5925300-  SRR6714636-  SRR6714637-  \
id                                                                              
chr14:100757429-100759901     3.249825     8.152294     0.831387     1.403842   
chr2:85062924-85065374        0.805493     1.028246     0.702153     0.845930   
chr21:46353613-46354147       2.171016     5.589816     0.946553     1.230527   
chr7:1607061-1607583       

In [23]:
result_file = "/home/galaxy/project/m6AQTL/2019_10_10/transcriptome_snp/SNPiR/shell/HeLa_cell/combined_results/results/MatrixEQTL_result/genotype_methy_boxplot_01_trend.txt"

with open(result_file, 'w') as fw:
    fw.write("SNP\tPeak\tgenotype_0\tgenotype_1\tgenotype_1\n")
    for snp, value_list in map_dict.items():
        peak,effect_size = value_list[0], str(value_list[1])
        values = df_geno.loc[snp, :].tolist()
        zero_list = [i for i in range(len(values)) if values[i] == 0]
        one_list = [i for i in range(len(values)) if values[i] == 1]
        two_list = [i for i in range(len(values)) if values[i] == 2]
        values = df_pheno.loc[peak, :].tolist()
        zero_list = [(values[x]) for x in zero_list]
        one_list = [(values[x]) for x in one_list]
        two_list = [(values[x]) for x in two_list]
        if float(effect_size) > 0 and (np.median(two_list) > np.median(one_list) > np.median(zero_list)):
            zero_list = [str(x) for x in zero_list]
            one_list = [str(x) for x in one_list]
            two_list = [str(x) for x in two_list]
            fw.write("\t".join([snp, peak, effect_size, ",".join(zero_list), ",".join(one_list), ",".join(two_list)]) + "\n")
        if float(effect_size) < 0 and (np.median(two_list) < np.median(one_list) < np.median(zero_list)):
            zero_list = [str(x) for x in zero_list]
            one_list = [str(x) for x in one_list]
            two_list = [str(x) for x in two_list]
            fw.write("\t".join([snp, peak, effect_size, ",".join(zero_list), ",".join(one_list), ",".join(two_list)]) + "\n")

In [21]:
# filter results based on effect size
np.median([1,1,1])

1.0